In [9]:
import pandas as pd

In [10]:
# Importation des données - on garde uniquement les colonnes et les dates qui nous intéressent

chunk_size = 1000000
file_path = "tr_mutualfunds S12.csv"
columns_to_keep = ['fdate', 'fundname', 'country', 'prc']

chunk_iterator = pd.read_csv(file_path, chunksize=chunk_size, usecols=columns_to_keep, dtype={'country': str})

processed_chunks = []

for i, chunk in enumerate(chunk_iterator):
    chunk_filtered = chunk[columns_to_keep]
    chunk_filtered['fdate'] = pd.to_datetime(chunk_filtered['fdate'], errors='coerce')
    chunk_filtered = chunk_filtered[(chunk_filtered['fdate'] >= '1975-01-01') & (chunk_filtered['fdate'] <= '2023-12-31') & (chunk_filtered['country'] == 'UNITED STATES')]

    processed_chunks.append(chunk_filtered)

data = pd.concat(processed_chunks)

In [11]:
def process_fund_group(group):

    group = group.sort_values('fdate')
    
    # On identifie la prochaine date attendue
    group['next_expected_date'] = group['fdate'] + pd.DateOffset(months=3)
    group['next_date'] = group['fdate'].shift(-1)
    valid_rows = (group['next_date'] == group['next_expected_date']) | group['next_date'].isna()
    valid_data = group.loc[valid_rows]

    # Calculer les rendements
    valid_data['prev_prc'] = valid_data['prc'].shift(1)
    valid_data['return'] = (valid_data['prc'] - valid_data['prev_prc']) / valid_data['prev_prc'] * 100
    
    return valid_data.dropna(subset=['return'])

In [12]:
# Calcul des rendements pour chaque fonds
data_m = data.groupby(['fdate', 'fundname']).agg({'prc': 'mean'}).reset_index()
result = data_m.groupby('fundname').apply(process_fund_group).reset_index(drop=True)

C:\Users\gabri\AppData\Local\Temp\ipykernel_109960\2387281292.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = data_m.groupby('fundname').apply(process_fund_group).reset_index(drop=True)


In [13]:
result.head()

,fdate,fundname,prc,next_expected_date,next_date,prev_prc,return
0,2016-03-31,1290 CONVERTIBLE SECURIT,67.100000,2016-06-30,2016-06-30,68.701111,-2.330546
1,2016-06-30,1290 CONVERTIBLE SECURIT,69.849091,2016-09-30,2016-09-30,67.100000,4.097006
2,2016-12-31,1290 CONVERTIBLE SECURIT,59.553571,2017-03-31,2017-03-31,69.849091,-14.739661
3,2017-03-31,1290 CONVERTIBLE SECURIT,57.646250,2017-06-30,2017-06-30,59.553571,-3.202699
4,2017-06-30,1290 CONVERTIBLE SECURIT,62.661250,2017-09-30,2017-09-30,57.646250,8.699612


In [14]:
# Création du dataframe final
result = result.drop(columns=['next_expected_date', 'next_date', 'prev_prc', 'prc'])

In [15]:
result.head()

,fdate,fundname,return
0,2016-03-31,1290 CONVERTIBLE SECURIT,-2.330546
1,2016-06-30,1290 CONVERTIBLE SECURIT,4.097006
2,2016-12-31,1290 CONVERTIBLE SECURIT,-14.739661
3,2017-03-31,1290 CONVERTIBLE SECURIT,-3.202699
4,2017-06-30,1290 CONVERTIBLE SECURIT,8.699612


In [16]:
result2 = result.groupby('fundname').filter(lambda x: x['return'].notnull().count() >= 20)
print(len(result["fundname"].unique()))

29152


In [17]:
output_file_path = 'mutual_funds_1975_2023.csv'
result2.to_csv(output_file_path, index=False)